<a href="https://colab.research.google.com/github/kshitijahande/Hate-Detection/blob/main/1_classifier_tfidf_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -qq ipynb
!pip install -qq plotly
!pip install -qq import-ipynb
!pip install -qq icecream
!pip install -qq transformers

!pip install -qq datasets


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import random as rn
from icecream import ic 
from torch.utils.data import Dataset, DataLoader
import os

In [17]:
from datasets import load_dataset

# ri = (datasets.ReadInstruction('train', to=10, unit='%') +
# datasets.ReadInstruction('train', from_=-80, unit='%'))
# dataset = datasets.load_dataset('hate_speech_offensive', split=ri)
dataset = load_dataset('hate_speech_offensive', split='train')


Using custom data configuration default
Reusing dataset hate_speech_offensive (/root/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5)


In [4]:
#print dataset properties
ic(dataset[1]['tweet'])
ic(dataset[1])
ic(dataset.column_names)
ic(dataset.num_rows)

# Task 1 - binary classification of hate=class:0 vs non-hate=class:1,2 
# "class":[
# 0:"hate speech"
# 1:"offensive language"
# 2:"neither"
# ]

ic| dataset[1]['tweet']: ('!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the '
                          '1st place!!')
ic| dataset[1]: {'class': 1,
                 'count': 3,
                 'hate_speech_count': 0,
                 'neither_count': 0,
                 'offensive_language_count': 3,
                 'tweet': '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe '
                          'in the 1st place!!'}
ic| dataset.column_names: ['count',
                           'hate_speech_count',
                           'offensive_language_count',
                           'neither_count',
                           'class',
                           'tweet']
ic| dataset.num_rows: 24783


24783

In [5]:
#shuffle and split train and test dataset randomly
train_test_split = dataset.train_test_split(shuffle=True)

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-d21fa33e190556a1.arrow and /root/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-0b5f0974b266240e.arrow


In [6]:
#get train and set data from original dataset
test = train_test_split.get('test')
train = train_test_split.get('train')


In [23]:
#convert to data frame
test_df = test.to_pandas()
train_full_df = train.to_pandas()

In [24]:
# split train dev/validation set
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_full_df, test_size=0.2)


In [29]:
# match total rows in each data frame
ic(train_df.shape[0])
ic(test_df.shape[0])
ic(dev_df.shape[0])
total = train_df.shape[0] + test_df.shape[0] + dev_df.shape[0]
ic(total)
ic(dataset.num_rows)

ic| train_df.shape[0]: 14869
ic| test_df.shape[0]: 6196
ic| dev_df.shape[0]: 3718
ic| total: 24783
ic| dataset.num_rows: 24783


24783

In [34]:
# hate vs non-hate distribution in data
train_df['class'].value_counts(ascending=True)


1    11475
2     2533
0      861
Name: class, dtype: int64

FEATURE GENERATION

In [39]:

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [40]:
# Use tf-idf vectorizer

other_exclusions = ["#ff", "ff", "rt"]
stopwords=stopwords = nltk.corpus.stopwords.words("english")
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def preprocess(text_string):
    """
    replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    standardized counts
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [45]:
train_df['tweet']

3776     RT @ahmadblack35: Can someone find the Georgia...
14042                             Where's my money, bitch?
13366                   @ReiddH i agree my fellow cracker.
1171     What you gonna do if she cheat ? Cry ? Fight f...
9215     &#8220;@__thaRealist: &#128530; dis hoe BRI wi...
                               ...                        
5356     @TooEffinCool lol I feel like a punk bitch sit...
3078                            These bitches crazzzyyyyyy
17987                       @SammyLightning_ Tim's a pussy
10388    RT @AzlanTheGoat: "These hoe ain't loyal"\n\n-...
3968     I'm still weak off the bitch gettin pissed on lol
Name: tweet, Length: 14869, dtype: object

In [46]:

# tfidf matrix and scores
tfidf = vectorizer.fit_transform(train_df['tweet']).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys:indices; values:IDF scores

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [47]:
tfidf.shape

(14869, 3028)

TASK 1 - TRAIN BINARY CLASSIFIER FOR HATE VS NON-HATE

In [49]:
X = pd.DataFrame(tfidf)
y = train_df['class'].astype(int)

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [57]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l2",C=0.01))
X_ = select.fit_transform(X,y)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [58]:
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_, y)

In [59]:
y_preds = model.predict(X_)


In [60]:
report = classification_report( y, y_preds )


In [61]:
print(report)


              precision    recall  f1-score   support

           0       0.57      0.76      0.65       861
           1       0.96      0.91      0.93     11475
           2       0.78      0.90      0.84      2533

    accuracy                           0.90     14869
   macro avg       0.77      0.86      0.81     14869
weighted avg       0.91      0.90      0.90     14869

